<h3>This notebook is a copy of the roi_detection notebook, however the NN here learns on augmented images.</h3>

Normally some of the augmentation could have been done on the fly, but for ROI detection NN I decided to go with the statically generated artifacts. It improved and stabilized the network's behavior slightly, so the model in this notebook is the final one in the family of custom cut ones.

In [1]:
import os
import absl.logging

from typing import Callable
from tensorflow import keras

from functions.ciou import ciou_loss, ciou_metric
from functions.loading_data import load_and_preprocess_data
from functions.loading_data import SMALLER_HEIGHT, SMALLER_WIDTH
from functions.plotting import plot_history

absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
get_names = lambda root_path: [
    file_name.split('.')[0]
    for dir_path, _, file_names in os.walk(root_path)
    for file_name in file_names
]
get_paths = lambda path: [f'{os.path.join(root, file)}' for root, dirs, files in os.walk(path) for file in files]
base_dir = os.path.join('..', 'data', 'images_original_inception_resnet_v2_200x150_splitted_with_augmentation_10p')
train_dir = os.path.join(base_dir, 'training')
valid_dir = os.path.join(base_dir, 'validation')

In [5]:
train_ds = load_and_preprocess_data(os.path.join('..', 'data', 'training_boxes.csv'), train_dir)
valid_ds = load_and_preprocess_data(os.path.join('..', 'data', 'validation_boxes.csv'), valid_dir)

In [8]:
def run_multi_attentive_model(
        model_factory: Callable,
        model_name: str,
        loss: Callable,
        metric: Callable,
        reduction_patience=5,
        monitor='val_root_ciou_metric'):
    MIN_DELTA = .001
    early_stopping = keras.callbacks.EarlyStopping(
        monitor=monitor,
        mode='max',
        patience=10,
        min_delta=MIN_DELTA)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor=monitor,
        mode='max',
        factor=0.95,
        min_delta=MIN_DELTA,
        patience=reduction_patience,
        min_lr=0.0005,
        verbose=1)
    model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath='models/' + model_name + '_{epoch}',
        save_best_only=True)
    tensor_board = keras.callbacks.TensorBoard(log_dir=f'tensor_logs/{model_name}')
    model = model_factory(loss, metric)

    return model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=500,
        callbacks=[reduce_lr, model_checkpoint, tensor_board, early_stopping])

In [9]:
def get_attentive_model2(loss: Callable, metric: Callable) -> keras.Model:
    def get_branch(_input: keras.layers.Layer, kernel_size: int) -> keras.layers.Layer:
        conv = keras.layers.Conv2D(64, kernel_size, padding='same', strides=2, activation='relu')(_input)
        conv = keras.layers.MaxPooling2D()(conv)

        return conv

    def get_inner_conv_module(prev: keras.layers.Layer, filters: int) -> keras.layers.Layer:
        conv_module = keras.layers.Conv2D(filters, 3, padding='same', activation='relu')(prev)
        conv_module = keras.layers.MaxPooling2D()(conv_module)
        conv_module = keras.layers.Dropout(.4)(conv_module)

        return conv_module

    def get_outer_conv_module(prev: keras.layers.Layer) -> keras.layers.Layer:
        conv_module = get_inner_conv_module(prev, 64)
        conv_module = get_inner_conv_module(conv_module, 128)
        conv_module = get_inner_conv_module(conv_module, 256)

        return conv_module

    def get_attention_module(prev: keras.layers.Layer) -> keras.layers.Layer:
        gap_layer = keras.layers.GlobalAveragePooling2D()(prev)
        gap_layer_res = keras.layers.Reshape((1, 1, 256))(gap_layer)
        dense = keras.layers.Dense(256, activation='relu')(gap_layer_res)
        dense = keras.layers.Dense(256, activation='softmax')(dense)
        mul_layer = keras.layers.Multiply()([prev, dense])

        return mul_layer

    def get_locator_module(prev: keras.layers.Layer) -> keras.layers.Layer:
        locator_module = keras.layers.Flatten()(prev)
        locator_module = keras.layers.Dense(256, activation='relu')(locator_module)
        locator_module = keras.layers.Dense(128, activation='relu')(locator_module)

        return locator_module

    _input = keras.layers.Input(shape=(SMALLER_HEIGHT, SMALLER_WIDTH, 3))
    branch1 = get_branch(_input, 3)
    branch2 = get_branch(_input, 5)
    branch3 = get_branch(_input, 7)
    merged_branches = keras.layers.concatenate([branch1, branch2, branch3])
    conv_module = get_outer_conv_module(merged_branches)
    attention_module1 = get_attention_module(conv_module)
    attention_module2 = get_attention_module(conv_module)
    merged_attentions = keras.layers.concatenate([attention_module1, attention_module2])
    locator_module = get_locator_module(merged_attentions)
    output = keras.layers.Dense(4, activation='sigmoid', name='root')(locator_module)
    gap_attention1 = keras.layers.GlobalAveragePooling2D()(attention_module1)
    gap_attention2 = keras.layers.GlobalAveragePooling2D()(attention_module2)
    aux_output = keras.layers.Dot(axes=1, normalize=True, name='dot')([gap_attention1, gap_attention2])
    model = keras.Model(_input, outputs=[output, aux_output])

    model.compile(
        optimizer='adam',
        loss={
            'root': loss,
            'dot': keras.losses.mean_squared_error,
        },
        loss_weights={'root': 1.0, 'dot': 0.1},
        metrics={
            'root': metric,
            'dot': keras.metrics.mean_squared_error
        })

    print(model.summary())

    return model

In [ ]:
history1 = run_multi_attentive_model(get_attentive_model2, f'roi_detection_augmmented_iou_1', ciou_loss, ciou_metric, 20)

In [12]:
plot_history(history1, metric_key='root_ciou_metric', val_metric_key='val_root_ciou_metric')